# Trabajo Integrador: DuckDB

## Instalación

In [ ]:
%pip install duckdb --upgrade

### Creando conexión

Nos permite establecer una conexión a una base de datos, por defecto, si no especificamos su nombre, la base de datos no persistirá y operará en memoria, por lo tanto no se almacenarán las tablas creadas. Trabajaremos en memoria ya que consideramos que es el fuerte de DUCKDB y su análisis es el pertinente de este trabajo de investigación. 

In [118]:
import duckdb as db

database = db.connect(database=":memory:")


##### Prueba con el dataset. En PostgreSQL almacenaremos archivos de órdenes y sus pagos. En AWS almacenaremos los productos y su categoría. El resto de archivos será almacenado de manera local, algunos en CSV y otros en PARQUET. Esta información se encuentra representada mediante un esquema en la documentación adjunta

In [ ]:
%pip install -q kagglehub        
import kagglehub, shutil, pathlib

path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")

local_csv_files = [
    "olist_customers_dataset.csv",
    "olist_geolocation_dataset.csv",
    "olist_order_items_dataset.csv",
    "olist_order_reviews_dataset.csv",
    "olist_sellers_dataset.csv",
    ## Estos archivos se almacenaran en PSQL 
    "olist_orders_dataset.csv",
    "olist_order_payments_dataset.csv", 
]

for file_name in local_csv_files:
    shutil.copy(f"{path}/{file_name}", f"dataset/{file_name}")

for file_name in local_csv_files:
    table = pathlib.Path(f"dataset/{file_name}").stem
    database.execute(f"""
        CREATE OR REPLACE TABLE {table} AS
        SELECT * FROM read_csv_auto('dataset/{file_name}');
    """)


#### Verificación básica de la creación de las tablas

In [ ]:
for file_name in local_csv_files:
    table = pathlib.Path(f"dataset/{file_name}").stem
    schema = database.sql(f"DESCRIBE {table}")
    print(schema)

#### Obteniendo los datos de forma remota - Conexión con AWS S3

**NOTA:** Para utilizar AWS S3 es necesario setear las credenciales para acceder al bucket, para eso debemos crear un user en la IAM de AWS, asignarle permisos y finalmente crear las claves de acceso para este usuario. Luego, estas credenciales son obtenidas desde un .env.
Es importante que la región del bucket y del usuario sean la misma.

In [ ]:
%pip install python-dotenv
from dotenv import load_dotenv
load_dotenv()  

import os

database.sql("INSTALL httpfs; LOAD httpfs;")

database.sql(f"""
SET s3_region='{os.getenv("AWS_REGION")}';
SET s3_access_key_id='{os.getenv("AWS_ACCESS_KEY_ID")}';
SET s3_secret_access_key='{os.getenv("AWS_SECRET_ACCESS_KEY")}';
""")

## Operaciones

### Creando las tablas con los datos de forma remota

In [122]:
database.sql(f"""
 CREATE OR REPLACE TABLE olist_products_dataset AS
        SELECT * FROM read_csv_auto('s3://ti-quadrelli-ribarov/olist_products_dataset.csv');
""")

database.sql(f"""
 CREATE OR REPLACE TABLE product_category_name_translation AS
        SELECT * FROM read_csv_auto('s3://ti-quadrelli-ribarov/product_category_name_translation.csv');
""")


#### Validación básica de la creación de las tablas con los datos remotos


In [ ]:
remote_csv_files  = [
    "olist_products_dataset.csv",
    "product_category_name_translation.csv",
]

for file_name in remote_csv_files:
    table = pathlib.Path(f"dataset/{file_name}").stem
    schema = database.sql(f"DESCRIBE {table}")
    print(schema)

#### Obteniendo los datos desde PostgreSQL

#### Nota: este paso se podría haber hecho sin DuckDB, mismo desde un manejador de base de datos como DBeaver o con la librería Pandas, sin embargo, nuevamente debido al objetivo de este proyecto, se eligió realizarlo utilizando DuckDB

In [ ]:

import os

# Esta celda utiliza duckdb para conectarse a PostgreSQL, crea la base de datos trabajo integrador
# en caso de que no exista 

database.execute("INSTALL postgres;")
database.execute("LOAD postgres;")

try:
    database.execute("DETACH pgadmin;")
except Exception:
    pass

conninfo = f"host={os.getenv('PG_HOST')} port={os.getenv('PG_PORT')} user={os.getenv('PG_USER')} password={os.getenv('PG_PASSWORD')} dbname=postgres"
database.execute(f"ATTACH '{conninfo}' AS pgadmin (TYPE postgres);")

exists = database.execute("""
    SELECT COUNT(*) > 0
    FROM postgres_query(
        'pgadmin',
        $$SELECT 1 FROM pg_database WHERE datname = 'trabajo_integrador'$$
    );
""").fetchone()[0]

if not exists:
    database.execute("""
        CALL postgres_execute(
            'pgadmin',
            $$CREATE DATABASE trabajo_integrador$$,
            use_transaction => false
        );
    """)

database.execute("DETACH pgadmin;")

In [ ]:
import pathlib, os

orders_csv   = pathlib.Path("olist_orders_dataset.csv")
payments_dataset = pathlib.Path("olist_order_payments_dataset.csv")
   

conninfo = f"host={os.getenv("PG_HOST")} port={os.getenv("PG_PORT")} user={os.getenv("PG_USER")} password={os.getenv("PG_PASSWORD")} dbname={os.getenv("PG_DB")}"
database.execute(f"ATTACH '{conninfo}' AS pgdb (TYPE postgres);")

#### Creación de tablas

In [ ]:
import pathlib

orders_csv = pathlib.Path(path) / "olist_orders_dataset.csv"
payments_dataset = pathlib.Path(path) / "olist_order_payments_dataset.csv"

database.execute(f"""
    DROP TABLE IF EXISTS pgdb.olist_orders;
    CREATE TABLE pgdb.olist_orders AS
    SELECT *
    FROM read_csv_auto('{orders_csv.as_posix()}', HEADER=TRUE);
""")

database.execute(f"""
    DROP TABLE IF EXISTS pgdb.olist_orders_payments;
    CREATE TABLE pgdb.olist_orders_payments AS
    SELECT *
    FROM read_csv_auto('{payments_dataset.as_posix()}', HEADER=TRUE);
""")

In [ ]:
postgreee_csv_files  = [
   "olist_orders", 
   "olist_orders_payments"
]

for file_name in postgreee_csv_files:
    table = pathlib.Path(f"dataset/{file_name}").stem
    schema = database.sql(f"DESCRIBE pgdb.{table}")
    print(schema)

## Exploracion del dataset

In [ ]:
database.sql("SHOW TABLES").df()

In [ ]:
database.sql("SELECT * FROM olist_customers_dataset LIMIT 5").df()

In [ ]:
print("Filas con nulos:")
database.sql("""
SELECT *
FROM olist_customers_dataset
WHERE
    customer_id IS NULL OR
    customer_unique_id IS NULL OR
    customer_zip_code_prefix IS NULL OR
    customer_city IS NULL OR
    customer_state IS NULL
LIMIT 5
""").df()

In [ ]:
database.sql("SELECT * FROM olist_geolocation_dataset LIMIT 5").df()

In [ ]:
tables = database.sql("SHOW TABLES").df()['name'].tolist()

for table in tables:
    print(f"Para la tabla: {table}")
    columns = database.sql(f"DESCRIBE {table}").df()['column_name'].tolist()
    where_clause = " OR ".join([f"{col} IS NULL" for col in columns])
    query = f"SELECT 1 FROM {table} WHERE {where_clause} LIMIT 1"
    result = database.sql(query).df()
    if not result.empty:
        print("Tiene nulos")
    else:
        print("NO tiene nulos")


In [ ]:
tablas_con_nulos = ["olist_order_reviews_dataset", "olist_orders_dataset", "olist_products_dataset", "olist_sellers_dataset", "olist_customers_dataset"]
for table in tablas_con_nulos:
    print(f"Tabla: {table}")
    columns = database.sql(f"DESCRIBE {table}").df()['column_name'].tolist()
    where_clause = " OR ".join([f"{col} IS NULL" for col in columns])
    query = f"SELECT * FROM {table} WHERE {where_clause} LIMIT 5"
    df_nulos = database.sql(query).df()
    print(df_nulos)


In [ ]:
total = database.sql("SELECT COUNT(*) AS total FROM olist_order_reviews_dataset").df().iloc[0]['total']
nulos = database.sql("SELECT COUNT(*) AS nulos FROM olist_order_reviews_dataset WHERE review_comment_title IS NULL").df().iloc[0]['nulos']
no_nulos = total - nulos
print(f"review_comment_title nulos: {nulos}")
print(f"review_comment_title no nulos: {no_nulos}")
print(f"Porcentaje de nulos: {nulos / total * 100:.2f}%")

In [ ]:
database.sql("DESCRIBE olist_orders_dataset").df()

In [ ]:
# Obtener la orden más vieja y la última según la columna order_purchase_timestamp
result = database.sql("""
    SELECT 
        MIN(order_purchase_timestamp) AS orden_mas_vieja,
        MAX(order_purchase_timestamp) AS orden_mas_reciente
    FROM olist_orders_dataset
""")
print(result)

In [ ]:
query = """
SELECT 
    EXTRACT(year FROM order_purchase_timestamp) AS anio,
    COUNT(*) AS cantidad_ordenes
FROM olist_orders_dataset
GROUP BY anio
ORDER BY cantidad_ordenes DESC
LIMIT 3
"""
database.sql(query).df()

In [ ]:
# Obtener la categoría de producto más solicitada
query = """
SELECT product_category_name, COUNT(*) AS cantidad
FROM olist_products_dataset
GROUP BY product_category_name
ORDER BY cantidad DESC
LIMIT 3
"""
database.sql(query).df()

In [ ]:
database.sql("""
SELECT customer_city, COUNT(*) AS cantidad
FROM olist_customers_dataset
GROUP BY customer_city
ORDER BY cantidad DESC
LIMIT 1
""").df()

In [ ]:
database.sql(
"""
SELECT seller_city, COUNT(*) AS cantidad_vendedores
FROM olist_sellers_dataset
GROUP BY seller_city
ORDER BY cantidad_vendedores DESC
LIMIT 1
"""
).df()

## Consultas

Esta sección consiste en realizar consultas interesantes que podrían ser de utilidad para un negocio que cuenta con el dataset en estudio

¿Cuántos días demora la entrega de las órdenes en los distintos años?

In [ ]:
query = """
SELECT 
    EXTRACT(year FROM order_purchase_timestamp) AS anio,
    AVG(DATEDIFF('day', order_purchase_timestamp, order_delivered_customer_date)) AS promedio_dias
FROM olist_orders_dataset
WHERE 
    order_delivered_customer_date IS NOT NULL
GROUP BY anio
ORDER BY anio DESC
"""
database.sql(query).df()

¿Cuántos clientes ordenaron en el sitio web?

In [ ]:
query = """
SELECT 
    EXTRACT(year FROM olist_orders_dataset.order_purchase_timestamp) AS anio,
    COUNT(DISTINCT olist_customers_dataset.customer_unique_id) AS cantidad_clientes
FROM olist_orders_dataset
JOIN olist_customers_dataset
    ON olist_orders_dataset.customer_id = olist_customers_dataset.customer_id
GROUP BY anio
ORDER BY anio
"""
database.sql(query).df()

¿Cuál fue el cliente con más órdenes?

In [ ]:
query = """
SELECT c.customer_unique_id, COUNT(*) AS n_ordenes
FROM olist_orders_dataset o
JOIN olist_customers_dataset c
  ON o.customer_id = c.customer_id
GROUP BY c.customer_unique_id
ORDER BY n_ordenes DESC
LIMIT 10;
"""
database.sql(query).df()

Como parte de un analisis, vamos a crear una tabla nueva con los clientes que realizaron más de 3 pedidos en un año y luego la consultaremos para saber si estos clientes siguieron realizando pedidos o abandonaron la plataforma

In [144]:
query ="""
CREATE TABLE clientes_pedidos_por_año AS
SELECT
    c.customer_unique_id,
    EXTRACT(YEAR FROM o.order_purchase_timestamp) AS año,
    COUNT(*) AS num_pedidos
FROM olist_orders_dataset o
JOIN olist_customers_dataset c
  ON o.customer_id = c.customer_id
GROUP BY c.customer_unique_id, año
ORDER BY c.customer_unique_id, año;
"""
database.sql(query)

In [ ]:
query = """
SELECT * 
FROM clientes_pedidos_por_año
"""
database.sql(query).df()

Una buena forma de saber por qué estos clientes dejaron la plataforma es observando sus reviews

In [146]:
query = """
CREATE OR REPLACE TABLE clientes_perdidos AS
WITH max_año AS (
    SELECT MAX(año) AS ultimo_año
    FROM clientes_pedidos_por_año
),
activos AS (
    SELECT
        cpa.customer_unique_id,
        cpa.año AS año_activo
    FROM clientes_pedidos_por_año cpa
    WHERE cpa.num_pedidos > 3
),
posteriores AS (
    SELECT
        a.customer_unique_id,
        MIN(cpa.año) AS primer_año_posterior
    FROM activos a
    JOIN clientes_pedidos_por_año cpa
      ON a.customer_unique_id = cpa.customer_unique_id
     AND cpa.año > a.año_activo
     AND cpa.num_pedidos > 0
    GROUP BY a.customer_unique_id, a.año_activo
)

SELECT
    a.customer_unique_id,
    a.año_activo
FROM activos a
JOIN max_año m
  ON 1=1
LEFT JOIN posteriores p
  ON a.customer_unique_id = p.customer_unique_id
 AND a.año_activo = p.primer_año_posterior - 1
WHERE
    a.año_activo < m.ultimo_año
  AND p.primer_año_posterior IS NULL;
"""
database.sql(query)

In [ ]:
query = """
SELECT *
FROM clientes_perdidos
"""
database.sql(query).df()

In [ ]:
query = """
SELECT 
    cp.customer_unique_id,
    cp.año_activo AS año,
    r.review_score AS puntaje,
    r.review_comment_message AS comentario
FROM clientes_perdidos cp
JOIN olist_customers_dataset c
    ON cp.customer_unique_id = c.customer_unique_id
JOIN olist_orders_dataset o
    ON c.customer_id = o.customer_id
LEFT JOIN olist_order_reviews_dataset r
    ON o.order_id = r.order_id
    AND EXTRACT(YEAR FROM o.order_purchase_timestamp) = cp.año_activo
WHERE r.review_comment_message IS NOT NULL
GROUP BY cp.customer_unique_id, cp.año_activo, r.review_score, r.review_comment_message
ORDER BY cp.customer_unique_id, cp.año_activo
"""
database.sql(query).df()


## Benchmark - En esta sección compararemos la performance de DuckDB contra su competidor más directo SQLite. Ambos utilizan a bases de datos embebidas.

### Realizaremos pruebas utilizando la base de datos almacenada en disco y en memoria.Las pruebas fueron realizadas en una Mac con M3 y 16GB de RAM. 

In [ ]:
# En esta celda obtendremos todas las tablas que están en la bdd. 
# Para luego obtener las que tienen más filas, más columnas y las más pesadas en tamaño.
# Utilizaremos estas tablas para realizar la comparación entre DuckDB y SQLite

all_tables = [r[0] for r in database.sql("SHOW TABLES").fetchall()]

table_data = []
# Se itera e las tablas, se obtienen los valores a medir y se guardan los resultados
for table in all_tables:
    rows_amount = database.sql(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    cols_amount = len(database.sql(f"PRAGMA table_info({table})").fetchall())
    csv = pathlib.Path("dataset") / f"{table}.csv"
    size = csv.stat().st_size if csv.exists() else 'En S3'
    table_data.append((table, rows_amount, cols_amount, size))

meta_df = pd.DataFrame(table_data, columns=["table", "rows", "cols", "size_bytes"])
# Las dos tablas que no están locales se ven en los resultados como 'En S3'
# Sin embargo, lo verificamos y su peso no es mayor a la tabla de geolocalización
display(meta_df)

meta_df["size_num"] = pd.to_numeric(meta_df["size_bytes"], errors="coerce")
meta_df["size_num"] = meta_df["size_num"].fillna(0)

most_rows = meta_df.loc[meta_df.rows.idxmax(), "table"]
most_cols = meta_df.loc[meta_df.cols.idxmax(), "table"]
most_size = meta_df.loc[meta_df.size_num.idxmax(), "table"]

benchmark_tables = {
    "más_filas":  most_rows,
    "más_cols":   most_cols,
    "csv_pesado": most_size,
}

print(f"Tablas elegidas para el benchmark: {benchmark_tables}")



## Ejecución en memoria

In [ ]:
import time, gc, statistics as stats
import  sqlite3

# Elegimos consultas basicas pero pesadas para evaluar el rendimiento de ambos motores
QUERIES = {
    # Cuenta el número de filas de la tabla
    "row_count" : "SELECT COUNT(*) FROM t",
    # Agrupa por una columna y cuenta el número de filas en cada grupo
    "group_all" : "SELECT 1 g, COUNT(*) FROM t GROUP BY 1",
    # Selecciona filas al azar
    "filter"    : "SELECT * FROM t WHERE random() < 0.01",
    # Realiza un join de la tabla consigo misma
    "self_join" : "SELECT a.* FROM t a JOIN t b ON a.rowid = b.rowid LIMIT 1000",
}

# Esta funcion ejecuta las consultas 5 veces y luego toma el tiempo para quedarse
# con el promedio de esos tiempos
def avg_time(factory, query_to_execute):                 
    times = []
    for i in range(5):
        # El modulo gc permite invocar funciones de garbage collector para que las pruebas sean
        # más "justas"
        gc.collect()

        # Esta fábrica se usara pa ejecutar la query con la BD nueva o la existente, 
        # de formma de poder variar las pruebas
        connection = factory()    

        # La funcion perf_counter permite medir el tiempo de ejecución                     
        initial_time = time.perf_counter()                
        query_to_execute(connection)

        # Se agrega el tiempo de ejecución. Tiempo inicial menos actual                           
        times.append(time.perf_counter() - initial_time)   

        # Si la conexión es de las cold, se cierra
        if getattr(connection, "_close_after_run", False):   
            connection.close()

    return stats.mean(times)

def add_row(rows, table, query, state, duck_sec, sqlite_sec):
    rows.append({
        "tabla": table,
        "consulta": query,
        "estado": state,                     
        "tiempo duckdb (ms)":  round(duck_sec   * 1_000, 3),
        "tiempo sqlite (ms)":  round(sqlite_sec * 1_000, 3),
    })

bench_rows = []

for table_characteristics, table in benchmark_tables.items():      
    df = database.sql(f"SELECT * FROM {table}").fetchdf()

    # Esta parte nos parecio un agregado interesante a testear.
    # Ya que, no es lo mismo que la conexión con la base de datps esté ya activa a tener que levantarla
    # desde 0 para ejecutar las consultas, entonces lo que hicimos fue crear una fábrica
    # que permita utilizar dos base de datos, una "fria" que se inicia cada vez que se ejcuta
    # la consulta y otra "caliente" que está siempre activa y se reutiliza.
    # Esto tambien permite ver si hay diferencias en los distitnos tipos de bases,
    # ya sea en cachés u otros datos que almacenen los motores para optimizar las consutlas.

    # Definición de bases "frias"

    cold_duck = lambda: duckdb.connect(":memory:")
    cold_sql  = lambda: sqlite3.connect(":memory:")
    cold_duck._close_after_run = True           
    cold_sql._close_after_run  = True          

    duck_ins_cold = avg_time(
        cold_duck,
        lambda c: (c.register("src", df),
                   c.execute("CREATE TABLE t AS SELECT * FROM src"))
    )
    sqlite_ins_cold = avg_time(
        cold_sql,
        lambda c: df.to_sql("t", c, if_exists="replace", index=False)
    )

    # Definición de bases "calientes"

    warm_duck = duckdb.connect(":memory:")       
    warm_sql  = sqlite3.connect(":memory:")      
    warm_duck.register("src", df)
    warm_duck.execute("CREATE TABLE t AS SELECT * FROM src")
    df.to_sql("t", warm_sql, if_exists="replace", index=False)

    duck_ins_warm = avg_time(lambda: warm_duck,
                             lambda c: c.execute("DELETE FROM t; INSERT INTO t SELECT * FROM src;"))
    sqlite_ins_warm = avg_time(lambda: warm_sql,
                               lambda c: (c.execute("DELETE FROM t"),
                                          df.to_sql("t", c, if_exists="append", index=False)))

    add_row(bench_rows, table_characteristics, "insert", "cold",
            duck_ins_cold, sqlite_ins_cold)
    add_row(bench_rows, table_characteristics, "insert", "warm",
            duck_ins_warm, sqlite_ins_warm)

    for query_name, query_to_exe in QUERIES.items():

        duck_q_cold = avg_time(
            cold_duck,
            lambda c: (c.register("src", df),
                       c.execute("CREATE TABLE t AS SELECT * FROM src"),
                       c.execute(query_to_exe).fetchall())
        )

        cold_sql_factory = lambda: sqlite3.connect(":memory:")
        cold_sql_factory._close_after_run = True

        sqlite_q_cold = avg_time(
            cold_sql_factory,
            lambda c: (
                df.to_sql("t", c, if_exists="replace", index=False),
                c.execute(query_to_exe).fetchall()        
            )
        )

        duck_q_warm = avg_time(lambda: warm_duck,
                               lambda c: c.execute(query_to_exe).fetchall())
        sqlite_q_warm = avg_time(lambda: warm_sql,
                                 lambda c: c.execute(query_to_exe).fetchall())    

        add_row(bench_rows, table_characteristics, query_name, "cold",
                duck_q_cold,   sqlite_q_cold)
        add_row(bench_rows, table_characteristics, query_name, "warm",
                duck_q_warm,   sqlite_q_warm)

    warm_duck.close(); warm_sql.close()

res_df = pd.DataFrame(bench_rows)

# Para que sea más visual, en caso que haya ganado DuckDB, se mostrará el valor positivo
# y en caso que haya ganado SQLite, se mostrará el valor negativo de la diferencia.
def diff(row):
    duckdb_time, sqlite_time = row["tiempo duckdb (ms)"], row["tiempo sqlite (ms)"]
    if duckdb_time < sqlite_time:                 
        return round(sqlite_time / duckdb_time, 2)
    else:                   
        return - round(duckdb_time / sqlite_time, 2)

res_df["diferencia (en veces más rápido)"] = res_df.apply(diff, axis=1)
display(res_df)

### Visualización de los resultados obtenidos

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

for query in res_df["consulta"].unique():
    sl = res_df[res_df["consulta"] == query]
    if sl.empty:
        continue

    sl = sl.sort_values(["tabla", "estado"])
    x = np.arange(len(sl) // 2)          
    width = 0.2

    fig, ax = plt.subplots(figsize=(8,4))

    cold_d = sl[sl["estado"]=="cold"]["tiempo duckdb (ms)"].values
    warm_d = sl[sl["estado"]=="warm"]["tiempo duckdb (ms)"].values
    cold_s = sl[sl["estado"]=="cold"]["tiempo sqlite (ms)"].values
    warm_s = sl[sl["estado"]=="warm"]["tiempo sqlite (ms)"].values

    ax.bar(x - 1.5*width, cold_d, width, label="DuckDB cold")
    ax.bar(x - 0.5*width, warm_d, width, label="DuckDB warm")
    ax.bar(x + 0.5*width, cold_s, width, label="SQLite cold")
    ax.bar(x + 1.5*width, warm_s, width, label="SQLite warm")

    ax.set_xticks(x)
    ax.set_xticklabels(sl["tabla"].unique(), rotation=45, ha="right")
    ax.set_ylabel("Milisegundos")
    ax.set_title(f"{query} - en memoria - mayor es peor")
    ax.legend()
    plt.tight_layout()
    plt.show()

## Ejecución en disco


In [ ]:
rows_disk = []

for key, tbl in benchmark_tables.items():
    df = database.sql(f"SELECT * FROM {tbl}").fetchdf()
    
    # Creamos archivos temporales para DuckDB y SQLite 
    # para que sea más "justo"
    
    duck_file   = pathlib.Path(tempfile.mktemp(suffix=".duckdb"))
    sqlite_file = pathlib.Path(tempfile.mktemp(suffix=".sqlite"))

    # inserción única
    duckdb.connect(duck_file).execute("CREATE TABLE t AS SELECT * FROM df").close()
    df.to_sql("t", sqlite3.connect(sqlite_file), if_exists="replace", index=False)

    # en este caso, no tiene sentido diferenciar entre conexión activa o no,
    # ya que estamos escribiendo en disco, por lo que no hay caché 

    duckdb_connection = duckdb.connect(duck_file)
    sqlite_connection = sqlite3.connect(sqlite_file)

    for qname, qsql in QUERIES.items():
        initial_time = time.perf_counter(); duckdb_connection.execute(qsql).fetchall(); duck_ms = (time.perf_counter()-initial_time)*1e3
        initial_time = time.perf_counter(); sqlite_connection.execute(qsql).fetchall(); sqlite_ms = (time.perf_counter()-initial_time)*1e3
        rows_disk.append({"tabla": key, "consulta": qname,
                          "tiempo duckdb (ms)": round(duck_ms,3),
                          "tiempo sqlite (ms)": round(sqlite_ms,3)})

    # Acá cerramos las conexiones como buena práctica, pero no es estrictamente necesario

    duckdb_connection.close(); sqlite_connection.close()
    duck_file.unlink(); sqlite_file.unlink()

res_disk = pd.DataFrame(rows_disk)

def diff(row):
    duckdb_time = row["tiempo duckdb (ms)"]
    sqlite_time = row["tiempo sqlite (ms)"]
    if duckdb_time < sqlite_time:                 
        return round(sqlite_time / duckdb_time, 2)
    else:                   
        return - round(duckdb_time / sqlite_time, 2)


res_disk["diferencia"] = res_disk.apply(diff, axis=1)

display(res_disk)

### Visualización de los resultados obtenidos

In [ ]:
for query in res_disk["consulta"].unique():
    sl = res_disk[res_disk["consulta"] == query].sort_values("tabla")
    x  = np.arange(len(sl))
    width = 0.3

    fig, ax = plt.subplots(figsize=(8,4))
    ax.bar(x - width/2, sl["tiempo duckdb (ms)"], width, label="DuckDB")
    ax.bar(x + width/2, sl["tiempo sqlite (ms)"], width, label="SQLite")

    ax.set_xticks(x)
    ax.set_xticklabels(sl["tabla"], rotation=45, ha="right")
    ax.set_ylabel("Milisegundos")
    ax.set_title(f"{query} - en disco - mayor es peor")
    ax.legend()
    plt.tight_layout()
    plt.show()